In [57]:
import numpy as np 
import matplotlib.pyplot as plt
import PIL.Image as Image
import gym
import random
import jdc
import pandas as pd

from gym import Env, spaces
import time

#get a matrix of distances between locations
df = pd.read_csv('dist_vologda_matrix.csv', sep=',')
df = df.iloc[:, 1:]

N = len(df.values[0])
df = df / 1000
df

,АЗС №35400 Кичменгский Городок,АЗС №35401 Великий Устюг-1,АЗС №35402 Великий Устюг-2,АЗС №35403 Дунай,АЗС №35404 Нюксеница,АЗС №35405 Борисово-Судское,АЗС №35406 Журавлево,АЗС №35407 Грязовец-1,АЗС №35408 Грязовец-2,АЗС №35409 Савино,...,АЗС №35454 Октябрьский пр.-1 (Вологда),АЗС №35455 Октябрьский пр.-2 (Вологда),АЗС №35456 Маяковская,АЗС №35457 Краснодонцев,АЗС №35458 Северсталь,АЗС №35459 Олимпийская,АЗС №35460 Северное шоссе-1,АЗС №35461 Северное шоссе-2,АЗС № 35462 Аксеново,Нефтебаза
0,0.0000,99.2272,101.8513,139.2135,154.2522,777.2635,102.4328,520.5020,517.8863,266.7014,...,610.3886,610.3028,607.3283,609.0579,611.0852,611.0605,604.2684,605.7555,502.9826,477.9325
1,99.0684,0.0000,4.9465,134.5201,149.5589,748.0318,5.5280,491.2703,488.6546,241.7276,...,581.1570,581.0711,578.0967,579.8263,581.8535,581.8288,575.0368,576.5238,473.7510,448.7008
2,101.8148,5.0688,0.0000,137.2665,152.3053,750.7781,3.0302,494.0166,491.4010,244.4739,...,583.9033,583.8174,580.8430,582.5726,584.5999,584.5752,577.7831,579.2702,476.4973,451.4472
3,139.2135,134.6791,137.3031,0.0000,15.0961,613.5690,137.8846,356.8075,354.1918,107.2648,...,446.6942,446.6083,443.6339,445.3635,447.3908,447.3660,440.5740,442.0610,339.2882,314.2380
4,154.2522,149.7178,152.3419,15.0961,0.0000,613.8947,152.9234,357.1332,354.5176,107.5905,...,447.0199,446.9341,443.9596,445.6892,447.7165,447.6918,440.8997,442.3868,339.6139,314.5638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,611.7861,582.6174,585.2414,447.9957,448.3214,191.2351,585.8229,181.8523,179.2367,345.2112,...,10.1276,10.0417,7.2967,3.3719,10.8241,0.0000,6.1623,4.6720,107.2118,142.1894
60,605.7962,576.6275,579.2515,442.0057,442.3315,185.0682,579.8330,175.8624,173.2467,339.2213,...,6.8726,6.7867,3.8122,4.8070,7.5691,6.8095,0.0000,1.5045,101.2218,136.1995
61,607.1467,577.9779,580.6020,443.3562,443.6820,186.4186,581.1835,177.2129,174.5972,340.5717,...,8.2230,8.1372,5.1627,3.5396,8.9196,5.5422,1.5229,0.0000,102.5723,137.5500
62,504.6641,475.4954,478.1194,340.8736,341.1994,274.3706,478.7009,74.7303,72.1146,238.0892,...,107.4957,107.4099,104.4354,106.1650,108.1923,108.1676,101.3755,102.8626,0.0000,35.0674


In [58]:
#custom gym environment for our problem
class TransportScape(Env):
    def __init__(self, env_config):
        self.num_of_trucks = N - 1
        self.num_of_locations = N - 1
        self.working_hours = 12 #working hours of each truck driver (harsh reality)
        self.truck_mean_speed = 60
        self.distances = df.values #an array, containing distances between locations

        self.action_space = spaces.Tuple((
            #i - truck number
            gym.spaces.Disctreet(self.num_of_trucks),
            #j - location number
            gym.spaces.Discreet(self.num_of_locations)
        ))
        self.observation_space = spaces.Dict(
        {
            #x_i - shows each truck's location
            'truck location': gym.spaces.Discreet(self.num_of_trucks),
            #p_j - shows whether a truck is assigned to this particular location or not
            'assignment': gym.spaces.Discreet(self.num_of_locations), 
            #u_i - shows whether a particular truck was used before
            'truck usage': gym.spaces.Discreet(self.num_of_trucks),
            #delta_i - shows how much time each truck has left until the end of the work day
            'time left': gym.spaces.Discreet(self.num_of_trucks)
        })
        self.state = {'truck location': np.full(self.num_of_trucks, self.num_of_locations - 1),
        'assignment': np.full(self.num_of_locations, False),
        'truck usage': np.full(self.num_of_trucks, False),
        'time left': np.full(self.num_of_trucks, self.working_hours)}

In [59]:
%%add_to TransportScape

def reset(self):
    self.state = {'truck location': np.full(self.num_of_trucks, self.num_of_locations - 1),
        'assignment': np.full(self.num_of_locations, False),
        'truck usage': np.full(self.num_of_trucks, False),
        'time left': np.full(self.num_of_trucks, self.working_hours)}
    return self.state

In [60]:
%%add_to TransportScape

def step(self, action):
    i, j = action[0], action[1]
    #if all locations have trucks assigned to them - end episode
    if np.all(self.state['assignment'] == True):
        self.done = True
    #if a truck is already assigned to this location - penalty
    if self.state['assignment'][j] == True:
        self.reward -= 100
    #if no time left - penalty
    if (self.state['time left'][i] - self.distances[self.state['truck location'][i], j] / self.truck_mean_speed - self.distances[self.state['truck location'][-1], j] / self.truck_mean_speed) < 0:
        self.reward -= 100
    #else - assign truck to a new location
    else:
        #if needs to use an unused before truck - penalty
        if self.state['truck usage'][i] == False:
            self.reward -= 100
            self.state['truck usage'][i] = True
        self.state['assignment'][i] = True
        self.reward -= self.distances[self.state['truck location'][i], j]
        self.state['time left'][action[0]] -= self.distances[self.state['truck location'][i], j] / self.truck_mean_speed
        self.state['truck location'][i] = j
    
    return self.state, self.reward, self.done, {}
